In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd
import numpy as np
from procedures import dump_live_config
from pure_funcs import (
    numpyize,
    denumpyize,
    live_config_dict_to_list_recursive_grid,
    sort_dict_keys,
    config_pretty_str,
    candidate_to_live_config,
)
import matplotlib.pyplot as plt
from collections import OrderedDict

In [ ]:
plt.rcParams["figure.figsize"] = [24, 13.5]
plt.rcParams["figure.facecolor"] = "w"
pd.set_option("display.precision", 10)

In [ ]:
def compute_pareto_front_mask(objectives, weights):
    """Compute a mask indicating whether each item in objectives is part of the Pareto front."""
    pareto_mask = [True] * len(objectives)  # Initialize all as True

    for i, individual_a in enumerate(objectives):
        for j, individual_b in enumerate(objectives):
            if i != j and dominates(individual_b, individual_a, weights):
                pareto_mask[i] = False
                break  # No need to check other individuals

    return pareto_mask


# Helper function to check domination
def dominates(individual_a, individual_b, weights):
    """Check if individual_a dominates individual_b."""
    better_in_any = False
    for a, b, w in zip(individual_a, individual_b, weights):
        if (w < 0 and a > b) or (
            w > 0 and a < b
        ):  # Minimize if weight is negative, maximize if positive
            return False
        elif a != b:
            better_in_any = True
    return better_in_any


def calc_dist(p0, p1):
    return ((p0[0] - p1[0]) ** 2 + (p0[1] - p1[1]) ** 2) ** 0.5

In [ ]:
# location of 'all_results.txt' file from multisymbol opt
with open("results_multi/2024-01-26T05_55_28_all_results.txt") as f:
    lines = [x.strip() for x in f.readlines()]
print(f"n backtests: {len(lines)}")

In [ ]:
xs = [json.loads(x) for x in lines if x]
res = pd.DataFrame(xs)
keys, weights = ["adg", "sharpe_ratio"], [1.0, 1.0]
worst_drawdown_threshold = 0.5
candidates = res[res.worst_drawdown <= worst_drawdown_threshold][keys]
pareto = candidates[compute_pareto_front_mask(candidates.values, weights)]

cands_norm = (candidates - candidates.min()) / (candidates.max() - candidates.min())
pareto_norm = cands_norm[compute_pareto_front_mask(cands_norm.values, weights)]
dists = [calc_dist(p, [1.0, 1.0]) for p in pareto_norm.values]
pareto_w_dists = pareto_norm.join(pd.Series(dists, name="dists", index=pareto_norm.index))
closest_to_ideal = pareto_w_dists.sort_values("dists")
best = closest_to_ideal.dists.idxmin()
print("best")
print(candidates.loc[best])
pareto.loc[closest_to_ideal.index]

In [ ]:
# scatterplot all candidates in blue, pareto front in orange, ideal target in green
plt.scatter(*candidates.values.T)
plt.scatter(*pareto.values.T)
plt.scatter(*pareto.max().values.T)
plt.scatter(*pareto.loc[best].T)

In [ ]:
# config from best result
res_sel = res.loc[best]
print(res_sel)
lc = res_sel.live_config
cfg = {"long": lc["long"], "short": lc["short"], "global": lc["global"]}
cfg["long"]["wallet_exposure_limit"] = lc["global"]["TWE_long"] / len(res_sel.symbols)
cfg["short"]["wallet_exposure_limit"] = lc["global"]["TWE_short"] / len(res_sel.symbols)
cfg["long"]["enabled"] = res_sel.long_enabled
cfg["short"]["enabled"] = res_sel.short_enabled
print(
    "cfg = ",
    json.dumps(denumpyize(cfg), indent=4, sort_keys=True).replace("true", "True").replace("false", "False"),
)
print("starting_balance =", res_sel.starting_balance)
print("symbols =", res.iloc[0].symbols)
